# Top of the book basis point calculation

This Calculator generates a signal from top of book spread for a configured venue in basis point. The Observed spot top of book spread value is smoothed by passing through a exponentially moving average (EMA) filter.

The (liquidity) signal provides a measurement of the current top of book spread - and tells us how tight the market spread of the venue is. 

This component should produce three values
- The basic top of the book spread in basis points: the observed instantaneous top of the book spread value is calculated then smoothed by passing through an exponential moving average (EMA) filter
- The value from 1) is normalized using a historical baseline. The resulting ratio is adjusted by two configurable parameters, referred to as the ratio signal 
- The value from 2) is floored by the desired minimum hourly normalized top of book spread ratio, referred to as the bounded ratio signal

## Inputs

* Order Book updates
* Trade Updates
* Market data status

## Filter Config

Rate of decay is determined by config param Order book to trade event ratio. The larger the ratio, the longer the filter window.



## Component config

- Instrument Id
- Venue Id
- order book to trade event ratio
- tob_spreadBasisPointBaseline
- hourlyMinimumSpreadBasisPointRatio
- dob_spreadBasisPointRatioDamp
- spreadRatioDampCcyAdjust

## Calc Triggers

* Order book updates
* Trades - only as a signal
* Mkt data status

## Calc 

Spot TOB basis point calc formula

$tob\_basis\_pt = 1e4 * \frac{(bestAsk - bestBid)} {(bestAsk + bestBid)/2}$

convert to basis pt or not is a choice

Above simple spread is passed through EMA filter. Output of the filter is the output of TOB basis point component.

The signal is updated and published each time a trade notification is received from a configured venue. When trade notification is received, the last order book received by the processor for the venue is used to calculate the spread.



$spreadRatio = \frac{tob\_spread\_basis\_pt}{tob\_spread\_basis\_pt\_baseline}$

$1.0 < spreadRatio < 10 * hourly\_min\_spread\_bp$

$spread\_ratio\_impact = spreadRatio^{ (tob\_spread\_bp\_ratio\_damp + spreadRatio\_damp\_ccy\_adjust)}$

spread_ratio_impact > hourly_min_spread_bp


## Steps

* Get top of the book quote
* calc tob_basis_pt
    * Check for negative spread
* Normalize spread
    * spreadSignal - Pass spread through EMA filter
    * ratio_floor - Get hourly parameter minimum spread basis point from config, ratio_floor
    * ratioSignal - max(1.0, min(max_liquidity_ratio_10 * ratio_floor, spreadSignal/spreadBaselineFromConfig_1))
    * Damped_Ratio - pow(ratioSignal, basisPtRatioDampFromConfig_0.98_1.12 + basisPtRatioDampCcyAdjust_0)
    * boundedRatioSignal - max(ratio_floor, Damped_Ratio)
* Publish - spreadSignal, ratioSignal, boundedRatioSignal

## Uses

* Quote Mid calc uses tob ratio signal to create a weight for scaling quote mid
* Spread Factors calculations
    * Competitors spread Calc
    * Max extra skew calc
    * TOB Spread factor
* Feed price calculator uses this to find whether there is significant midPrice change. whether threshold > 0.1
* Market readiness - just availablility check not the actual value
* Used in Mid uncertainity calc - midUncertainity + 0.5 * tobSprdBasisPt

